# Introduction
This project applies regression techniques to predict future housing market values for Fairfax County, VA. This area has experienced rapid development in the past and it would be insightful to see what the future could hold for this county, and by extension, the greater DC Metropolitan Area.

### Preparing the data
All data used for this project was obtained from the [Fairfax County website's demographic reports](https://www.fairfaxcounty.gov/demographics/reports). It spans the years from 1991 to 2018 and includes the estimated population, number of households, the number of housing units, median household income, and the median market value for each year. <br>
<font size="1"> **_Note that some data for the median household income is missing and will be dealt with appropriately_**</font>

In [39]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Importing dataset
df = pd.read_csv('https://rahbarprojects.s3.amazonaws.com/fairfax_county_housing.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, 5].values

# Filling in missing data
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imp = imp.fit(y[:, 0:0])
y[:, 0:0] = imp.transform(y[:, 0:0])

C:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


IndexError: too many indices for array

# Analysis/Modeling
Do work here

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here